In [15]:
import pandas as pd
import numpy as np
####

Country = "Hungary"
Country_abbr = "Hun" # Pol, Hun
text_name = "7279372069"+".txt"

####


df = pd.read_csv("../data/" + Country + '_DataFrame.csv')
''' Select data frame that ends at 2020-03 '''
end_index = None
for index in range(len(df)):
    if df.iloc[index, 0] == "2020-03":
        end_index = index
        break
assert (end_index is not None)


''' Scale the data '''
def scale_series(series, mean, std):
    return (series - mean)/std
for col_num in range(1, len(df.columns)):
    df.iloc[:, col_num] = scale_series(df.iloc[:, col_num], df.iloc[:end_index+1, col_num ].mean(), df.iloc[:end_index+1, col_num ].std())


In [16]:

file_path = "./Output/" + Country_abbr +"/corr/"


dict_weight = dict()

with open(file_path+text_name, "r") as f:
    is_Weight = False
    line = f.readline()
    while len(line.strip()):
        if line.strip().startswith("_") or line.strip().startswith("-"):
            line = f.readline()
            continue
        if is_Weight:
            
            name, weight = line.strip().split(":")
            name = name.strip()
            weight = weight.strip()
            weight = float(weight)
            dict_weight[name] = weight
        else:
            if line.startswith("Weight"):
                is_Weight = True
        line = f.readline()


In [18]:

df = df[["Date",*dict_weight.keys()]].dropna().reset_index()


In [20]:

FCI = np.matmul(df[dict_weight.keys()],np.square(list(dict_weight.values())))
Date = df["Date"]
df_output = pd.DataFrame({"Date":Date, "FCI":FCI})

end_index = None
for index in range(len(df)):
    if df_output.iloc[index, 0] == "2020-03":
        end_index = index
        break
assert (end_index is not None)
for col_num in range(1, len(df_output.columns)):
    df_output.iloc[:, col_num] = scale_series(df_output.iloc[:, col_num], df_output.iloc[:end_index+1, col_num ].mean(), df_output.iloc[:end_index+1, col_num ].std())

df_output.to_csv("Output/" + Country_abbr+"/FCI.csv", index = False)
df_output.to_csv("PCA results for visualization/"+ Country + "/FCI.csv", index = False)
